# Data for Interactive Dash Maps

In [14]:
import pandas as pd

import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import functions.functions_data
import functions.functions_mask_wearing

from flask import Flask
import os

pd.set_option('display.max_columns', None) # To display all columns

### Reading in data

In [3]:
dfs_country = functions.functions_data.get_data("data/CMU_Global_data/Full_Survey_Data/country/smooth/", "country")

Read in data completed.


In [4]:
countries = pd.concat(dfs_country, ignore_index=True)

In [5]:
countries = functions.functions_data.insert_month(countries)

Created month column.


In [15]:
countries = functions.functions_mask_wearing.deal_with_NaNs_masks(countries)

NaNs before update: 165309
NaNs after update: 0
Updated.


<ipython-input-15-1896cf284c31>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  countries = functions.functions_mask_wearing.deal_with_NaNs_masks(countries)


### Adjusting data for app.py - very basic first implementation

In [16]:
df = countries[countries["age_bucket"]=="overall"]
df = df[df["date"]=="2020-07-16"]
df = df.groupby(["country_agg", "GID_0","gender", "date"])[["rolling_total_responses","smoothed_pct_cli_weighted"]].mean()

In [17]:
dff = df.stack().reset_index()

In [18]:
dff = dff.rename(columns={"level_4": "data_cat", 0: "amount"})

In [19]:
dff.to_csv("dash_data/countries_dash.csv.gzip", compression="gzip")

### Adjusting data for app.py - more complicated

In [35]:
df1 = countries[countries["date"]=="2020-07-16"]

In [36]:
df1 = countries.groupby(["country_agg", "GID_0","gender", "age_bucket", "date"])[["rolling_total_responses",
            "smoothed_pct_cli_weighted", "smoothed_pct_worked_outside_home_weighted", 
              "smoothed_pct_grocery_outside_home_weighted", "smoothed_pct_ate_outside_home_weighted", 
              "smoothed_pct_spent_time_with_non_hh_weighted", "smoothed_pct_attended_public_event_weighted", 
              "smoothed_pct_used_public_transit_weighted", "smoothed_pct_direct_contact_with_non_hh_weighted", 
              "smoothed_pct_no_public_weighted", "smoothed_pct_wear_mask_all_time_weighted", 
            "smoothed_pct_wear_mask_most_time_weighted", "smoothed_pct_wear_mask_half_time_weighted", 
            "smoothed_pct_wear_mask_some_time_weighted", "smoothed_pct_wear_mask_none_time_weighted" ]].mean()

In [37]:
df2 = df1.stack().reset_index()

In [38]:
df2 = df2.rename(columns={"level_5": "data_cat", 0: "amount"})

In [39]:
df2.to_csv("dash_data/countries_dash2.csv.gzip", compression="gzip")

# Basic Map

In [34]:
fig = px.choropleth(
        data_frame = dff2,
        locations="GID_0",
        color="amount",
        hover_name="country_agg",
        color_continuous_scale=px.colors.sequential.BuPu)
fig.show("browser")